In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.driver.maxResultSize":"0"
         },
 "driverMemory": "100000M"
}

In [2]:
sc.install_pypi_package("pandas==0.24.2")
sc.install_pypi_package("boto3")
sc.install_pypi_package("s3fs")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
35,application_1668092583877_0036,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/db/83/7d4008ffc2988066ff37f6a0bb6d7b60822367dcb36ba5e39aa7801fda54/pandas-0.24.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/85/ac/92f998fc52a70afd7f6b788142632afb27cd60c8c782d1452b7466603332/pytz-2022.6-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/5a/fd/d814f9cbefebbea88977628d11b860b5d564ba6f16f64c378e2da2a36405/boto3-1.17.112-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/d0/693477c688348654ddc21dcdce0817653a294aa43f41771084c25e7ff9c7/s3transfer-0.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c7/ea/11c3beca131920f552602b98d7ba9fc5b46bee6a59cbd48a95a85cbb8f41/botocore-1.20.112-py2.py3-none-any.whl
  Found existing installation: botocore 1.18.6
    Not uninstalling botocore at /usr/lib/python2.7/site-packages, outside environment /tmp/1668109489710-0
  Found existing installation: s3transfe

In [3]:
confs = ["spark.yarn.executor.memoryOverhead","spark.sql.shuffle.partitions"]
print(sc._conf.get('spark.driver.memory'))
print(sc._conf.get('spark.app.name'))
print(sc._conf.get('spark.submit.deployMode'))
print(sc._conf.get("spark.yarn.executor.memoryOverhead"))
print(sc._conf.get("spark.sql.files.maxPartitionBytes"))
print(sc._conf.get("spark.pyspark.virtualenv.enabled"))
print(sc._conf.get("spark.driver.maxResultSize"))
for conf in confs:
    print(conf,":",sc._conf.get(conf))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100000M
livy-session-35
client
None
None
true
0
('spark.yarn.executor.memoryOverhead', ':', None)
('spark.sql.shuffle.partitions', ':', None)

In [4]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth, to_date
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, MapType, IntegerType
from pyspark.sql.types import *
from pyspark.sql import functions as F
from collections import Counter
import pyspark
import pandas as pd
import re
import s3fs
import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# schema = StructType([ StructField("theme", StringType(), True),StructField("theme_category", ArrayType(StringType()), True)])
adjacency_list = spark.read.option("header",True).csv('s3://atharva-dev/adjacency_list.csv')
adjacency_list = adjacency_list.toPandas()
adjacency_list['topics'] = adjacency_list['topics'].apply(lambda x: x.split(','))
adjacency_list = spark.createDataFrame(adjacency_list)
matrix_df = spark.read.option("header",True).csv('s3://atharva-dev/GKG themes tagged with ESG.csv')
matrix_df = matrix_df.toPandas()
theme_categories = [x.lower().replace(' ','_') for x in list([x.strip() for x in matrix_df.columns])]
theme_categories.remove('theme')
schema = StructType([ StructField("theme", StringType(), True),StructField("theme_category", StringType(), True)])
theme_dict_df = spark.read.option("header",True).csv('s3://atharva-dev/theme_dict.csv',schema=schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
adjacency_list.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|               theme|              topics|
+--------------------+--------------------+
|    act_harmthreaten|[fclt_risk, fclt_...|
|   act_makestatement|[fclt_government,...|
|         agriculture|[bbk_topic_enviro...|
|    aid_humanitarian|[bbk_topic_social...|
|            alliance|  [fclt_affiliation]|
|         appointment|    [fclt_employees]|
|       armedconflict|[bbk_topic_govern...|
|       assassination|[fclt_government,...|
|           austerity|  [fclt_affiliation]|
|   aviation_incident|[fclt_health_&_sa...|
|        black_market|     [fclt_customer]|
|            bullying|[fclt_environment...|
|           ceasefire|[fclt_health_&_sa...|
|charasmatic_leade...|[bbk_topic_govern...|
|        claim_credit|[fclt_health_&_sa...|
|      constitutional| [fclt_human_rights]|
|          corruption|[bbk_topic_govern...|
|crime_common_robbery|[fclt_health_&_sa...|
| crime_illegal_drugs|[bbk_topic_social...|
|crisislex_c01_chi...|[bbk_topic

In [7]:
theme_dict_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|               theme|      theme_category|
+--------------------+--------------------+
|          tax_fncact|bbk_topic_environ...|
|        media_social|    bbk_topic_social|
|           education|bbk_subcategory_e...|
|soc_pointsofinterest|           fclt_risk|
|       tax_ethnicity|bbk_category_sust...|
|    tax_worldmammals|fclt_labor_relations|
|           media_msm|bbk_category_envi...|
|            alliance|bbk_subcategory_u...|
|         tax_weapons|fclt_labor_relations|
|         agriculture|bbk_subcategory_b...|
|      soc_innovation|    fclt_affiliation|
|  tax_worldlanguages|bbk_subcategory_s...|
|    movement_general|bbk_category_sust...|
|                kill|    bbk_topic_social|
|              affect|bbk_subcategory_h...|
|    natural_disaster|bbk_topic_environ...|
|manmade_disaster_...|       fclt_supplier|
|       tax_worldfish|           fclt_risk|
|             protest|bbk_subcategory_p...|
|              strike|bbk_subcat

In [8]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('atharva-dev')
def get_folder_name(path):
    return path.split('/')[0]

files = [obj.key for obj in my_bucket.objects.all()]
folders = list(dict.fromkeys(list(map(get_folder_name, files))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/tmp/1668109489710-0/lib/python2.7/site-packages/boto3/compat.py:86: PythonDeprecationWarning: Boto3 will no longer support Python 2.7 starting July 15, 2021. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.6 or later. More information can be found here: https://aws.amazon.com/blogs/developer/announcing-end-of-support-for-python-2-7-in-aws-sdk-for-python-and-aws-cli-v1/
  warnings.warn(warning, PythonDeprecationWarning)

In [ ]:
for folder in folders:
    print(folder)
    #Read all parquet files and concatenate them in one dataframe from a folder
    df = spark.read.parquet("s3://atharva-dev/"+folder)
    #Sort by date
    df = df.sort('date')
    #Keep only date and not time from date column
    df = df.withColumn('date', F.to_date(df.date))
    #For themes column in df, only keep the keys of dictionary in array
    df = df.withColumn('themes', F.explode('themes'))
    #From themes column, get the theme and store in same column
    df = df.withColumn('themes', F.col('themes').getItem('theme'))
    #From tone column, get the tone and store in same column
    df = df.withColumn('tone', F.col('tone').getItem('tone'))
    #Remove excess spaces from all columns
    df = df.withColumn('themes', F.trim(F.col('themes')))
    df = df.withColumn('tone', F.trim(F.col('tone')))
    #lowercase and remove spaces from themes
    df = df.withColumn('themes', F.lower(F.col('themes')))
    #join df with adjacency list on themes
    df = df.join(adjacency_list, df.themes == adjacency_list.theme, how='left')
    #Drop theme column
    df = df.drop('theme')
    # Create a dataset comprising of rows with topics having null values into df_2 and drop them from df
    df_2 = df.filter(df.topics.isNull())
    df = df.filter(df.topics.isNotNull())
    #Join df_2 with theme_dict_df on themes
    df_2 = df_2.join(theme_dict_df, df_2.themes == theme_dict_df.theme, how='left')
    #Drop topics and theme column from df_2, and rename theme_category to topics
    df_2 = df_2.drop('topics','theme').withColumnRenamed('theme_category','topics')
    #Drop rows with null values in topics column
    df_2 = df_2.filter(df_2.topics.isNotNull())
    #Explode topics column in df
    df = df.withColumn('topics', F.explode('topics'))
    #Concatenate df and df_2 and sort by date
    df = df.union(df_2).sort('date')
    #Pivot topics column to get two columns per topic with topic + _tone with mean aggregate of tone and topic + _freq with count aggregate of topics
    df = df.groupBy('date','country').pivot('topics').agg(F.mean('tone'),F.count('topics'))
    # df.coalesce(1).write.csv("s3://atharva-dev/"+folder+".csv",header=True, mode='overwrite')
    #Write spark dataframe to csv file without coalescing
    df.write.csv("s3://atharva-dev/"+folder+".csv",header=True, mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…